In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, scrolledtext
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
import csv
from PIL import Image, ImageTk
import os
try:
    import customtkinter as ctk
    USE_CTK = True
except Exception:
    USE_CTK = False


In [2]:
class KarateClubAnalyzer:
    def __init__(self):
        # Initialisation
        self.G = None  # Graphe
        self.adjacency_matrix = None  # Matrice d'adjacence
        self.analysis_results = {}  # Résultats d'analyse
        self.node_colors = {}  # Couleurs des nœuds
        self.create_karate_network()  # Création du réseau
    
    def create_karate_network(self):
        """Crée manuellement le réseau du club de karaté de Zachary"""
        # Initialisation du graphe
        self.G = nx.Graph()
        self.G.add_nodes_from(range(34))  # Ajout des 34 nœuds
        
        # Définition des arêtes
        edges = [
            (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8),
            (0, 10), (0, 11), (0, 12), (0, 13), (0, 17), (0, 19), (0, 21), (0, 31),
            (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30),
            (2, 3), (2, 7), (2, 8), (2, 9), (2, 13), (2, 27), (2, 28), (2, 32),
            (3, 7), (3, 12), (3, 13), (4, 6), (4, 10), (5, 6), (5, 10), (5, 16),
            (6, 16), (8, 30), (8, 32), (8, 33), (9, 33), (13, 33), (14, 32), (14, 33),
            (15, 32), (15, 33), (18, 32), (18, 33), (19, 33), (20, 32), (20, 33),
            (22, 32), (22, 33), (23, 25), (23, 27), (23, 29), (23, 32), (23, 33),
            (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (27, 33),
            (28, 31), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33),
            (32, 33)
        ]
        
        # Ajout des arêtes et création de la matrice d'adjacence
        self.G.add_edges_from(edges)
        self.create_adjacency_matrix()
        self.reset_node_colors()  # Réinitialisation des couleurs
    
    def reset_node_colors(self):
        # Réinitialise les couleurs des nœuds
        for node in self.G.nodes():
            self.node_colors[node] = '#2E86AB'  # Couleur par défaut
    
    def create_adjacency_matrix(self):
        # Crée la matrice d'adjacence
        n = len(self.G.nodes())
        self.adjacency_matrix = np.zeros((n, n), dtype=int)  # Initialisation
        for i, j in self.G.edges():
            self.adjacency_matrix[i][j] = 1  # Arête présente
            self.adjacency_matrix[j][i] = 1  # Matrice symétrique
    
    def calculate_basic_properties(self):
        # Calcule les propriétés de base du graphe
        properties = {
            'order': len(self.G.nodes()),  # Ordre
            'size': len(self.G.edges()),  # Taille
            'density': nx.density(self.G),  # Densité
            'average_degree': sum(dict(self.G.degree()).values()) / len(self.G.nodes()),  # Degré moyen
            'diameter': nx.diameter(self.G) if nx.is_connected(self.G) else 'Disconnected',  # Diamètre
            'average_path_length': nx.average_shortest_path_length(self.G) if nx.is_connected(self.G) else 'Disconnected'  # Longueur moyenne des chemins
        }
        return properties
    
    def degree_distribution(self):
        # Distribution des degrés
        degrees = [deg for node, deg in self.G.degree()]  # Liste des degrés
        max_degree = max(degrees) if degrees else 0  # Degré maximum
        distribution = {}
        for deg in range(0, max_degree + 1):
            distribution[deg] = degrees.count(deg)  # Comptage par degré
        return distribution
    
    def clustering_analysis(self):
        # Analyse du clustering
        clustering = {
            'average_clustering': nx.average_clustering(self.G),  # Clustering moyen
            'node_clustering': nx.clustering(self.G),  # Clustering par nœud
            'transitivity': nx.transitivity(self.G)  # Transitivité
        }
        return clustering
    
    def centrality_measures(self):
        # Mesures de centralité
        centralities = {
            'degree_centrality': nx.degree_centrality(self.G),  # Centralité de degré
            'betweenness_centrality': nx.betweenness_centrality(self.G),  # Centralité d'intermédiarité
            'closeness_centrality': nx.closeness_centrality(self.G),  # Centralité de proximité
            'eigenvector_centrality': nx.eigenvector_centrality(self.G, max_iter=1000)  # Centralité de vecteur propre
        }
        return centralities
    
    def find_most_central_nodes(self, centralities, top_k=5):
        # Trouve les nœuds les plus centraux
        most_central = {}
        for centrality_name, centrality_dict in centralities.items():
            # Tri décroissant par valeur de centralité
            sorted_nodes = sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)[:top_k]
            most_central[centrality_name] = sorted_nodes
        return most_central
    
    def clique_analysis(self):
        # Analyse des cliques
        all_cliques = list(nx.find_cliques(self.G))  # Toutes les cliques maximales
        clique_number = max(len(clique) for clique in all_cliques) if all_cliques else 0  # Nombre de clique
        cliques = {
            'all_cliques': all_cliques,  # Liste des cliques
            'clique_number': clique_number,  # Taille de la plus grande clique
            'largest_cliques': [clique for clique in all_cliques if len(clique) == clique_number]  # Plus grandes cliques
        }
        return cliques
    
    def k_core_analysis(self):
        # Analyse des k-cores
        return nx.core_number(self.G)  # Numéro de core pour chaque nœud
    
    def detect_communities(self):
        """Détecte les communautés et retourne deux groupes"""
        try:
            # Détection des communautés par modularité
            communities = nx.community.greedy_modularity_communities(self.G)
            communities_list = list(communities)
            if len(communities_list) >= 2:
                group1 = list(communities_list[0])  # Premier groupe
                group2 = list(communities_list[1])  # Deuxième groupe
                # Fusion des communautés supplémentaires dans le groupe 2
                for comm in communities_list[2:]:
                    group2.extend(list(comm))
            else:
                # Méthode de secours : division des nœuds en deux
                nodes = list(self.G.nodes())
                mid = len(nodes) // 2
                group1 = nodes[:mid]
                group2 = nodes[mid:]
            return group1, group2
        except:
            # Méthode de secours en cas d'erreur
            nodes = list(self.G.nodes())
            mid = len(nodes) // 2
            return nodes[:mid], nodes[mid:]
    
    def run_complete_analysis(self):
        # Exécute l'analyse complète
        centralities = self.centrality_measures()
        self.analysis_results = {
            'basic_properties': self.calculate_basic_properties(),  # Propriétés de base
            'degree_distribution': self.degree_distribution(),  # Distribution des degrés
            'clustering': self.clustering_analysis(),  # Clustering
            'centralities': centralities,  # Mesures de centralité
            'most_central_nodes': self.find_most_central_nodes(centralities),  # Nœuds les plus centraux
            'cliques': self.clique_analysis(),  # Cliques
            'k_cores': self.k_core_analysis(),  # K-cores
        }
        return self.analysis_results


In [3]:
class ModernKarateClubGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("A.R.S Project - Analyse du Réseau Karaté Club")
        self.root.geometry("1200x700")
        self.root.configure(bg='#2C3E50')
        
        # Background image
        self.bg_image = None
        self.bg_label = None
        bg_path = os.path.join(os.getcwd(), 'background.jpg')
        if os.path.exists(bg_path):
            try:
                img = Image.open(bg_path)
                img = img.resize((1200, 700), Image.LANCZOS)
                self.bg_image = ImageTk.PhotoImage(img)
                self.bg_label = tk.Label(self.root, image=self.bg_image)
                self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
            except Exception:
                pass
        
        self.analyzer = KarateClubAnalyzer()
        self.analyzer.run_complete_analysis()
        
        # Current display state
        self.current_display = "graph"
        
        # Export checkboxes
        self.analysis_checkboxes = {}
        self.graph_checkboxes = {}
        self.matrix_checkboxes = {}
        
        # Prefer rounded controls if available
        self.use_ctk = USE_CTK
        
        self.setup_gui()
        self.show_graph()
    
    def make_button(self, parent, **kwargs):
        if self.use_ctk:
            kwargs_ctk = dict(text=kwargs.get('text', ''), command=kwargs.get('command'))
            btn = ctk.CTkButton(parent, corner_radius=44, fg_color='#3498DB', hover_color='#2980B9',
                                text_color='white', font=('Segoe UI', 12, 'bold'), **{})
            # set text/command after creation for clarity
            btn.configure(**kwargs_ctk)
            return btn
        return tk.Button(parent, relief=tk.FLAT, bg='#3498DB', fg='white', activebackground='#2980B9',
                         activeforeground='white', font=('Segoe UI', 12, 'bold'), **kwargs)
    
    def make_frame(self, parent, **kwargs):
        if self.use_ctk:
            return ctk.CTkFrame(parent, corner_radius=44, fg_color=kwargs.get('bg', '#34495E'))
        return tk.Frame(parent, **kwargs)
    
    def setup_gui(self):
        container_bg = '#2C3E50'
        main_container = tk.Frame(self.root, bg=container_bg)
        main_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=12)
        
        header_frame = tk.Frame(main_container, bg=container_bg)
        header_frame.pack(fill=tk.X, pady=(0, 14))
        
        logo_frame = tk.Frame(header_frame, bg=container_bg)
        logo_frame.pack(side=tk.LEFT)
        
        logo_path = os.path.join(os.getcwd(), 'Logo.png')
        if os.path.exists(logo_path):
            try:
                logo_img = Image.open(logo_path)
                logo_img = logo_img.resize((48, 48), Image.LANCZOS)
                self.logo_photo = ImageTk.PhotoImage(logo_img)
                tk.Label(logo_frame, image=self.logo_photo, bg=container_bg).pack(side=tk.LEFT, padx=(0, 12))
            except Exception:
                pass
        
        tk.Label(logo_frame, text="Université d'Alger 1", font=('Segoe UI', 14, 'bold'), 
                 fg='white', bg=container_bg).pack(side=tk.LEFT)
        
        # Theme title color adjusted
        tk.Label(header_frame, text="A.R.S Project", font=('Segoe UI', 28, 'bold'), 
                 fg='#FFFFFF', bg=container_bg).pack(side=tk.LEFT, expand=True)
        
        content_frame = tk.Frame(main_container, bg=container_bg)
        content_frame.pack(fill=tk.BOTH, expand=True)
        
        nav_frame = self.make_frame(content_frame, bg='#34495E')
        nav_frame.configure(width=240)
        nav_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 18))
        if not self.use_ctk:
            nav_frame.pack_propagate(False)
        
        # Navigation buttons (added Analyse de réseau)
        buttons = [
            ("Graph", self.show_graph),
            ("Distribution des degrés", self.show_degree_distribution),
            ("Analyse de réseau", self.show_analysis),
            ("Modifier", self.show_modifier),
            ("Exporter", self.show_exporter),
        ]
        for text, cmd in buttons:
            btn = self.make_button(nav_frame, text=text, command=cmd)
            if self.use_ctk:
                btn.pack(fill='x', padx=14, pady=8)
            else:
                btn.configure(width=20, height=2)
                btn.pack(fill=tk.X, padx=14, pady=8)
        
        # Display area: remove grey panel look (no border, dark bg so image dominates around widgets)
        self.display_frame = tk.Frame(content_frame, bg=container_bg, bd=0, relief=tk.FLAT)
        self.display_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
    
    def clear_display(self):
        for widget in self.display_frame.winfo_children():
            widget.destroy()
    
    def show_graph(self, show_groups=False):
        self.clear_display()
        self.current_display = "graph"
        
        # Create control frame
        control_frame = tk.Frame(self.display_frame, bg='#2C3E50')
        control_frame.pack(fill=tk.X, padx=8, pady=4)
        
        # Toggle button for groups
        toggle_btn = self.make_button(control_frame, text="Afficher avec Groupes" if not show_groups else "Afficher sans Groupes", 
                                      command=lambda: self.show_graph(not show_groups))
        toggle_btn.pack(side=tk.LEFT, padx=4)
        
        fig = Figure(figsize=(10, 8), dpi=100)
        ax = fig.add_subplot(111)
        
        pos = nx.spring_layout(self.analyzer.G, seed=42)
        
        if show_groups:
            group1, group2 = self.analyzer.detect_communities()
            node_colors = []
            for node in self.analyzer.G.nodes():
                if node in group1:
                    node_colors.append('#E74C3C')  # Red for group 1
                else:
                    node_colors.append('#3498DB')  # Blue for group 2
            
            nx.draw(self.analyzer.G, pos, ax=ax, with_labels=True, 
                    node_color=node_colors, node_size=700, font_size=10, 
                    edge_color='gray', alpha=0.7, font_weight='bold')
            
            ax.set_title("Réseau du Club de Karaté de Zachary - Groupes", fontsize=16, fontweight='bold')
            
            # Show group information
            info_frame = tk.Frame(self.display_frame, bg='#34495E', relief=tk.RAISED, bd=2)
            info_frame.pack(fill=tk.X, padx=8, pady=4)
            
            group1_frame = tk.Frame(info_frame, bg='#E74C3C', relief=tk.RAISED, bd=1)
            group1_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=4, pady=4)
            tk.Label(group1_frame, text=f"Groupe 1: {sorted(group1)}", 
                    font=('Segoe UI', 10, 'bold'), bg='#E74C3C', fg='white').pack(pady=2)
            
            group2_frame = tk.Frame(info_frame, bg='#3498DB', relief=tk.RAISED, bd=1)
            group2_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=4, pady=4)
            tk.Label(group2_frame, text=f"Groupe 2: {sorted(group2)}", 
                    font=('Segoe UI', 10, 'bold'), bg='#3498DB', fg='white').pack(pady=2)
        else:
            node_colors = [self.analyzer.node_colors[node] for node in self.analyzer.G.nodes()]
            nx.draw(self.analyzer.G, pos, ax=ax, with_labels=True, 
                    node_color=node_colors, node_size=700, font_size=10, 
                    edge_color='gray', alpha=0.7, font_weight='bold')
            ax.set_title("Réseau du Club de Karaté de Zachary", fontsize=16, fontweight='bold')
        
        ax.axis('off')
        
        canvas = FigureCanvasTkAgg(fig, master=self.display_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=4, pady=4)
        
        toolbar = NavigationToolbar2Tk(canvas, self.display_frame)
        toolbar.update()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def show_degree_distribution(self):
            self.clear_display()
            self.current_display = "degree_distribution"
            
            fig = Figure(figsize=(10, 8), dpi=100)
            ax = fig.add_subplot(111)
            
            distribution = self.analyzer.analysis_results['degree_distribution']
            degrees = list(distribution.keys())
            counts = list(distribution.values())
            
            bars = ax.bar(degrees, counts, alpha=0.7, color='#3498DB', edgecolor='#2980B9')
            ax.set_xlabel('Degré', fontsize=12, fontweight='bold')
            ax.set_ylabel('Nombre de nœuds', fontsize=12, fontweight='bold')
            ax.set_title('Distribution des Degrés', fontsize=16, fontweight='bold')
            ax.grid(True, alpha=0.3)
            
            # Set x-axis to show integer values for degrees
            ax.set_xticks(degrees)
            ax.set_xticklabels(degrees)
            
            # Set y-axis to show integer values for counts
            max_count = max(counts) if counts else 0
            y_ticks = range(0, max_count + 1)
            ax.set_yticks(y_ticks)
            ax.set_yticklabels(y_ticks)
            
            for bar in bars:
                height = bar.get_height()
                if height > 0:
                    ax.text(bar.get_x() + bar.get_width()/2., height,
                            f'{int(height)}', ha='center', va='bottom')
            
            canvas = FigureCanvasTkAgg(fig, master=self.display_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=4, pady=4)
            
            toolbar = NavigationToolbar2Tk(canvas, self.display_frame)
            toolbar.update()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def show_analysis(self):
        self.clear_display()
        self.current_display = "analysis"
        
        title = tk.Label(self.display_frame, text="Analyse de réseau", 
                         font=('Segoe UI', 20, 'bold'), bg='#2C3E50', fg='white')
        title.pack(pady=(6, 8))
        
        # Button frame with wrapping
        button_frame = tk.Frame(self.display_frame, bg='#2C3E50')
        button_frame.pack(fill=tk.X, padx=8, pady=4)
        
        analysis_buttons = [
            ("Propriétés Basiques", self.show_basic_properties),
            ("Distribution des Degrés", self.show_degree_dist),
            ("Coefficients de Clustering", self.show_clustering),
            ("Mesures de Centralité", self.show_centrality),
            ("Nœuds les Plus Centraux", self.show_most_central),
            ("Analyse des Cliques", self.show_cliques),
            ("Analyse des K-Cores", self.show_k_cores),
            ("Matrice d'Adjacence", self.show_adjacency_matrix),
        ]
        
        # Wrap buttons in two rows (4 buttons per row)
        row1 = tk.Frame(button_frame, bg='#2C3E50')
        row1.pack(fill=tk.X, pady=2)
        row2 = tk.Frame(button_frame, bg='#2C3E50')
        row2.pack(fill=tk.X, pady=2)
        
        for i, (text, cmd) in enumerate(analysis_buttons):
            btn = self.make_button(row1 if i < 4 else row2, text=text, command=cmd)
            if self.use_ctk:
                btn.pack(side=tk.LEFT, padx=4, pady=2, fill='x', expand=True)
            else:
                btn.configure(height=1)
                btn.pack(side=tk.LEFT, padx=4, pady=2, fill=tk.X, expand=True)
        
        # Content area
        self.analysis_content = tk.Frame(self.display_frame, bg='#34495E', relief=tk.RAISED, bd=2)
        self.analysis_content.pack(fill=tk.BOTH, expand=True, padx=8, pady=4)
        
        # Show default view
        self.show_basic_properties()
    
    def clear_analysis_content(self):
        for widget in self.analysis_content.winfo_children():
            widget.destroy()
    
    def show_basic_properties(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Propriétés Basiques", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        # Create scrollable table container
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        bp = self.analyzer.analysis_results['basic_properties']
        
        # Header with fixed column widths
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X)
        
        col1_width = 30
        col2_width = 30
        tk.Label(header_frame, text="Propriété", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col1_width, anchor='w').pack(side=tk.LEFT, padx=10, pady=8)
        tk.Label(header_frame, text="Valeur", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col2_width, anchor='w').pack(side=tk.LEFT, padx=10, pady=8)
        
        # Rows with same column widths
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, (key, value) in enumerate(bp.items()):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=str(key).replace('_', ' ').title(), 
                    font=('Segoe UI', 11), bg=row_bg[i % 2], width=col1_width, anchor='w').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(row, text=str(value), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col2_width, anchor='w').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_degree_dist(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Distribution des Degrés", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        dist = self.analyzer.analysis_results['degree_distribution']
        
        # Header with fixed column widths
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X)
        
        col1_width = 20
        col2_width = 25
        tk.Label(header_frame, text="Degré", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        tk.Label(header_frame, text="Nombre de Nœuds", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        
        # Rows with same column widths
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, (deg, count) in enumerate(sorted(dist.items())):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=str(deg), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(row, text=str(count), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_clustering(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Coefficients de Clustering", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        cl = self.analyzer.analysis_results['clustering']
        
        # Summary frame
        summary_frame = tk.Frame(table_container, bg='#3498DB', relief=tk.RAISED, bd=2)
        summary_frame.pack(fill=tk.X, pady=(0, 10))
        tk.Label(summary_frame, text=f"Coefficient moyen: {cl['average_clustering']:.4f}", 
                font=('Segoe UI', 12, 'bold'), bg='#3498DB', fg='white').pack(side=tk.LEFT, padx=20, pady=8)
        tk.Label(summary_frame, text=f"Transitivité: {cl['transitivity']:.4f}", 
                font=('Segoe UI', 12, 'bold'), bg='#3498DB', fg='white').pack(side=tk.LEFT, padx=20, pady=8)
        
        # Scrollable table
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Header with fixed column widths
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X)
        
        col1_width = 20
        col2_width = 25
        tk.Label(header_frame, text="Nœud", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        tk.Label(header_frame, text="Coefficient", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        
        # Rows with same column widths
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, (node, coeff) in enumerate(sorted(cl['node_clustering'].items())):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=str(node), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(row, text=f"{coeff:.4f}", font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_centrality(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Mesures de Centralité", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        cent = self.analyzer.analysis_results['centralities']
        nodes = sorted(self.analyzer.G.nodes())
        
        # Scrollable table
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Header with fixed column widths
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X)
        
        headers = ["Nœud"] + [h.replace('_', ' ').title() for h in cent.keys()]
        col_width = 18
        for h in headers:
            tk.Label(header_frame, text=h, font=('Segoe UI', 11, 'bold'), 
                    bg='#3498DB', fg='white', width=col_width, anchor='c').pack(side=tk.LEFT, padx=5, pady=8)
        
        # Rows with same column widths
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, node in enumerate(nodes):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=str(node), font=('Segoe UI', 10), 
                    bg=row_bg[i % 2], width=col_width, anchor='c').pack(side=tk.LEFT, padx=5, pady=4)
            for centrality in cent.values():
                val = centrality.get(node, 0)
                tk.Label(row, text=f"{val:.4f}", font=('Segoe UI', 10), 
                        bg=row_bg[i % 2], width=col_width, anchor='c').pack(side=tk.LEFT, padx=5, pady=4)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_most_central(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Nœuds les Plus Centraux", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        most_central = self.analyzer.analysis_results['most_central_nodes']
        
        col1_width = 12
        col2_width = 18
        col3_width = 22
        
        for centrality_name, nodes_list in most_central.items():
            # Section title
            section_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
            section_frame.pack(fill=tk.X, pady=(10, 5))
            tk.Label(section_frame, text=centrality_name.replace('_', ' ').title(), 
                    font=('Segoe UI', 12, 'bold'), bg='#3498DB', fg='white').pack(padx=10, pady=6)
            
            # Table header
            header_frame = tk.Frame(scrollable_frame, bg='#2980B9', relief=tk.RAISED, bd=1)
            header_frame.pack(fill=tk.X)
            tk.Label(header_frame, text="Rang", font=('Segoe UI', 11, 'bold'), 
                    bg='#2980B9', fg='white', width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(header_frame, text="Nœud", font=('Segoe UI', 11, 'bold'), 
                    bg='#2980B9', fg='white', width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(header_frame, text="Valeur", font=('Segoe UI', 11, 'bold'), 
                    bg='#2980B9', fg='white', width=col3_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            
            # Rows with same column widths
            row_bg = ['#FFFFFF', '#F8F9FA']
            for i, (node, value) in enumerate(nodes_list):
                row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
                row.pack(fill=tk.X)
                tk.Label(row, text=str(i+1), font=('Segoe UI', 11), 
                        bg=row_bg[i % 2], width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
                tk.Label(row, text=str(node), font=('Segoe UI', 11), 
                        bg=row_bg[i % 2], width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
                tk.Label(row, text=f"{value:.4f}", font=('Segoe UI', 11), 
                        bg=row_bg[i % 2], width=col3_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_cliques(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Analyse des Cliques", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        clq = self.analyzer.analysis_results['cliques']
        
        # Summary
        summary_frame = tk.Frame(table_container, bg='#3498DB', relief=tk.RAISED, bd=2)
        summary_frame.pack(fill=tk.X, pady=(0, 10))
        tk.Label(summary_frame, text=f"Nombre total de cliques: {len(clq['all_cliques'])}", 
                font=('Segoe UI', 12, 'bold'), bg='#3498DB', fg='white').pack(side=tk.LEFT, padx=20, pady=8)
        tk.Label(summary_frame, text=f"Nombre de clique: {clq['clique_number']}", 
                font=('Segoe UI', 12, 'bold'), bg='#3498DB', fg='white').pack(side=tk.LEFT, padx=20, pady=8)
        
        # Scrollable table
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Header
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X, pady=(0, 5))
        tk.Label(header_frame, text="Cliques Maximales", font=('Segoe UI', 12, 'bold'), 
                bg='#3498DB', fg='white').pack(padx=10, pady=8)
        
        col1_width = 20
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, clique in enumerate(clq['largest_cliques']):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=f"Clique {i+1}", font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col1_width, anchor='w').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(row, text=str(sorted(clique)), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], anchor='w').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_k_cores(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Analyse des K-Cores", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        kc = self.analyzer.analysis_results['k_cores']
        
        # Header with fixed column widths
        header_frame = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_frame.pack(fill=tk.X)
        
        col1_width = 20
        col2_width = 25
        tk.Label(header_frame, text="Nœud", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        tk.Label(header_frame, text="Valeur K-Core", font=('Segoe UI', 12, 'bold'), 
                 bg='#3498DB', fg='white', width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=8)
        
        # Rows with same column widths
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i, (node, k_value) in enumerate(sorted(kc.items())):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            tk.Label(row, text=str(node), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col1_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
            tk.Label(row, text=str(k_value), font=('Segoe UI', 11), 
                    bg=row_bg[i % 2], width=col2_width, anchor='c').pack(side=tk.LEFT, padx=10, pady=6)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
    def show_adjacency_matrix(self):
        self.clear_analysis_content()
        
        title = tk.Label(self.analysis_content, text="Matrice d'Adjacence", 
                         font=('Segoe UI', 16, 'bold'), bg='#34495E', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(self.analysis_content, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        h_scrollbar = ttk.Scrollbar(table_container, orient="horizontal", command=canvas.xview)
        v_scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(xscrollcommand=h_scrollbar.set, yscrollcommand=v_scrollbar.set)
        
        n = len(self.analyzer.G.nodes())
        matrix = self.analyzer.adjacency_matrix
        
        # Header row with node numbers
        header_row = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_row.pack(fill=tk.X)
        tk.Label(header_row, text="", font=('Segoe UI', 10, 'bold'), 
                bg='#3498DB', fg='white', width=5, anchor='c').pack(side=tk.LEFT, padx=3, pady=5)
        for j in range(n):
            tk.Label(header_row, text=str(j), font=('Segoe UI', 9, 'bold'), 
                    bg='#3498DB', fg='white', width=4, anchor='c').pack(side=tk.LEFT, padx=2, pady=5)
        
        # Matrix rows
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i in range(n):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            # Row label
            tk.Label(row, text=str(i), font=('Segoe UI', 9, 'bold'), 
                    bg=row_bg[i % 2], width=5, anchor='c').pack(side=tk.LEFT, padx=3, pady=3)
            # Matrix values
            for j in range(n):
                val = int(matrix[i][j])
                tk.Label(row, text=str(val), font=('Segoe UI', 9), 
                        bg=row_bg[i % 2], width=4, anchor='c').pack(side=tk.LEFT, padx=2, pady=3)
        
        canvas.pack(side="left", fill="both", expand=True)
        v_scrollbar.pack(side="right", fill="y")
        h_scrollbar.pack(side="bottom", fill="x")
    
    def show_modifier(self):
        self.clear_display()
        self.current_display = "modifier"
        
        title = tk.Label(self.display_frame, text="Modification du Réseau", 
                         font=('Segoe UI', 20, 'bold'), bg='#2C3E50', fg='white')
        title.pack(pady=12)
        
        mod_frame = tk.Frame(self.display_frame, bg='#2C3E50')
        mod_frame.pack(expand=True)
        
        mod_buttons = [
            ("Ajouter Nœud", self.add_node),
            ("Supprimer Nœud", self.remove_node),
            ("Ajouter Arête", self.add_edge),
            ("Supprimer Arête", self.remove_edge),
            ("Réinitialiser Graphe", self.reset_network),
        ]
        for text, cmd in mod_buttons:
            btn = self.make_button(mod_frame, text=text, command=cmd)
            if self.use_ctk:
                btn.pack(pady=8)
            else:
                btn.configure(width=22, height=2)
                btn.pack(pady=8)
    
    def show_exporter(self):
        self.clear_display()
        self.current_display = "exporter"
        
        title = tk.Label(self.display_frame, text="Exportation des Données", 
                         font=('Segoe UI', 20, 'bold'), bg='#2C3E50', fg='white')
        title.pack(pady=8)
        
        switch_frame = tk.Frame(self.display_frame, bg='#2C3E50')
        switch_frame.pack(fill=tk.X, padx=10)
        
        content_holder = tk.Frame(self.display_frame, bg='#2C3E50')
        content_holder.pack(fill=tk.BOTH, expand=True, padx=10, pady=6)
        
        self.analysis_section = tk.Frame(content_holder, bg='#2C3E50')
        self.graphs_section = tk.Frame(content_holder, bg='#2C3E50')
        self.matrix_section = tk.Frame(content_holder, bg='#2C3E50')
        
        self.setup_analysis_tab(self.analysis_section)
        self.setup_graphs_tab(self.graphs_section)
        self.setup_matrix_tab(self.matrix_section)
        
        def show_section(section):
            for f in (self.analysis_section, self.graphs_section, self.matrix_section):
                f.pack_forget()
            section.pack(fill=tk.BOTH, expand=True)
        
        self.make_button(switch_frame, text="Analyse", command=lambda: show_section(self.analysis_section)).pack(side=tk.LEFT, padx=6, pady=6)
        self.make_button(switch_frame, text="Graphs", command=lambda: show_section(self.graphs_section)).pack(side=tk.LEFT, padx=6, pady=6)
        self.make_button(switch_frame, text="Matrice", command=lambda: show_section(self.matrix_section)).pack(side=tk.LEFT, padx=6, pady=6)
        
        show_section(self.analysis_section)
        
        confirm_btn = self.make_button(self.display_frame, text="Confirmer l'Export", command=self.export_selected)
        if self.use_ctk:
            confirm_btn.configure(font=('Segoe UI', 14, 'bold'), fg_color='#27AE60', hover_color='#1E8449')
        else:
            confirm_btn.configure(font=('Segoe UI', 14, 'bold'), bg='#27AE60', activebackground='#1E8449')
        confirm_btn.pack(side=tk.RIGHT, padx=16, pady=8)
    
    def setup_analysis_tab(self, parent):
        for child in list(parent.children.values()):
            child.destroy()
        
        analysis_options = [
            ("Propriétés Basiques", "basic_properties"),
            ("Distribution des Degrés", "degree_distribution"),
            ("Coefficients de Clustering", "clustering"),
            ("Mesures de Centralité", "centralities"),
            ("Nœuds les Plus Centraux", "most_central_nodes"),
            ("Analyse des Cliques", "cliques"),
            ("Analyse des K-Cores", "k_cores"),
        ]
        
        for text_label, key in analysis_options:
            var = self.analysis_checkboxes.get(key) or tk.BooleanVar()
            cb = tk.Checkbutton(parent, text=text_label, variable=var, 
                                font=('Segoe UI', 12), bg='#2C3E50', fg='white', selectcolor='#2C3E50', activebackground='#2C3E50')
            cb.pack(anchor=tk.W, padx=8, pady=4)
            self.analysis_checkboxes[key] = var
    
    def setup_graphs_tab(self, parent):
        for child in list(parent.children.values()):
            child.destroy()
        
        graph_options = [
            ("Graphe du Réseau", "network_graph"),
            ("Distribution des Degrés", "degree_graph"),
        ]
        
        for text_label, key in graph_options:
            var = self.graph_checkboxes.get(key) or tk.BooleanVar()
            cb = tk.Checkbutton(parent, text=text_label, variable=var,
                                font=('Segoe UI', 12), bg='#2C3E50', fg='white', selectcolor='#2C3E50', activebackground='#2C3E50')
            cb.pack(anchor=tk.W, padx=8, pady=4)
            self.graph_checkboxes[key] = var
    
    def setup_matrix_tab(self, parent):
        for child in list(parent.children.values()):
            child.destroy()
        
        preview_btn = self.make_button(parent, text="Aperçu de la Matrice", command=self.preview_matrix)
        preview_btn.pack(pady=6)
        
        var = self.matrix_checkboxes.get("adjacency_matrix") or tk.BooleanVar()
        cb = tk.Checkbutton(parent, text="Matrice d'Adjacence", variable=var,
                            font=('Segoe UI', 12), bg='#2C3E50', fg='white', selectcolor='#2C3E50', activebackground='#2C3E50')
        cb.pack(anchor=tk.W, padx=8, pady=4)
        self.matrix_checkboxes["adjacency_matrix"] = var
    
    def preview_matrix(self):
        preview_window = tk.Toplevel(self.root)
        preview_window.title("Aperçu de la Matrice d'Adjacence")
        preview_window.geometry("1000x700")
        preview_window.configure(bg='#2C3E50')
        
        title = tk.Label(preview_window, text="Matrice d'Adjacence", 
                         font=('Segoe UI', 18, 'bold'), bg='#2C3E50', fg='white')
        title.pack(pady=10)
        
        table_container = tk.Frame(preview_window, bg='#ECF0F1')
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        canvas = tk.Canvas(table_container, bg='#ECF0F1')
        h_scrollbar = ttk.Scrollbar(table_container, orient="horizontal", command=canvas.xview)
        v_scrollbar = ttk.Scrollbar(table_container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg='#ECF0F1')
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(xscrollcommand=h_scrollbar.set, yscrollcommand=v_scrollbar.set)
        
        n = len(self.analyzer.G.nodes())
        matrix = self.analyzer.adjacency_matrix
        
        # Header row with node numbers
        header_row = tk.Frame(scrollable_frame, bg='#3498DB', relief=tk.RAISED, bd=2)
        header_row.pack(fill=tk.X)
        tk.Label(header_row, text="", font=('Segoe UI', 10, 'bold'), 
                bg='#3498DB', fg='white', width=5, anchor='c').pack(side=tk.LEFT, padx=3, pady=5)
        for j in range(n):
            tk.Label(header_row, text=str(j), font=('Segoe UI', 9, 'bold'), 
                    bg='#3498DB', fg='white', width=4, anchor='c').pack(side=tk.LEFT, padx=2, pady=5)
        
        # Matrix rows
        row_bg = ['#FFFFFF', '#F8F9FA']
        for i in range(n):
            row = tk.Frame(scrollable_frame, bg=row_bg[i % 2], relief=tk.FLAT)
            row.pack(fill=tk.X)
            # Row label
            tk.Label(row, text=str(i), font=('Segoe UI', 9, 'bold'), 
                    bg=row_bg[i % 2], width=5, anchor='c').pack(side=tk.LEFT, padx=3, pady=3)
            # Matrix values
            for j in range(n):
                val = int(matrix[i][j])
                tk.Label(row, text=str(val), font=('Segoe UI', 9), 
                        bg=row_bg[i % 2], width=4, anchor='c').pack(side=tk.LEFT, padx=2, pady=3)
        
        canvas.pack(side="left", fill="both", expand=True)
        v_scrollbar.pack(side="right", fill="y")
        h_scrollbar.pack(side="bottom", fill="x")
    
    def export_selected(self):
        export_dir = filedialog.askdirectory(title="Sélectionner le dossier d'export")
        if not export_dir:
            return
        
        try:
            for key, var in self.analysis_checkboxes.items():
                if var.get():
                    self.export_analysis(key, export_dir)
            for key, var in self.graph_checkboxes.items():
                if var.get():
                    self.export_graph(key, export_dir)
            if self.matrix_checkboxes.get("adjacency_matrix") and self.matrix_checkboxes["adjacency_matrix"].get():
                self.export_matrix(export_dir)
            messagebox.showinfo("Succès", "Exportation terminée avec succès!")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de l'export: {str(e)}")
    
    def export_analysis(self, analysis_type, export_dir):
        filename = f"{export_dir}/analyse_{analysis_type}.csv"
        
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            if analysis_type == 'basic_properties':
                writer.writerow(["PROPRIETES BASIQUES"])
                for key, value in self.analyzer.analysis_results['basic_properties'].items():
                    writer.writerow([key, value])
            elif analysis_type == 'degree_distribution':
                writer.writerow(["DISTRIBUTION DES DEGRES"])
                writer.writerow(["Degre", "Nombre de noeuds"])
                distribution = self.analyzer.analysis_results['degree_distribution']
                for degree, count in distribution.items():
                    writer.writerow([degree, count])
            elif analysis_type == 'clustering':
                writer.writerow(["COEFFICIENTS DE CLUSTERING"])
                clustering = self.analyzer.analysis_results['clustering']
                writer.writerow(["Coefficient de clustering moyen:", clustering['average_clustering']])
                writer.writerow(["Transitivite:", clustering['transitivity']])
                writer.writerow([])
                writer.writerow(["Noeud", "Coefficient de clustering"])
                for node, coeff in clustering['node_clustering'].items():
                    writer.writerow([node, coeff])
            elif analysis_type == 'centralities':
                writer.writerow(["MESURES DE CENTRALITE"])
                centralities = self.analyzer.analysis_results['centralities']
                nodes = list(self.analyzer.G.nodes())
                writer.writerow(["Noeud"] + list(centralities.keys()))
                for node in nodes:
                    row = [node]
                    for centrality in centralities.values():
                        row.append(f"{centrality.get(node, 0):.6f}")
                    writer.writerow(row)
            elif analysis_type == 'most_central_nodes':
                writer.writerow(["NOEUDS LES PLUS CENTRAUX"])
                most_central = self.analyzer.analysis_results['most_central_nodes']
                for centrality_name, nodes in most_central.items():
                    writer.writerow([centrality_name.replace('_', ' ').title() + ":"])
                    for node, value in nodes:
                        writer.writerow([f"  Noeud {node}", f"{value:.4f}"])
                    writer.writerow([])
            elif analysis_type == 'cliques':
                writer.writerow(["ANALYSE DES CLIQUES"])
                cliques = self.analyzer.analysis_results['cliques']
                writer.writerow(["Nombre de cliques maximales:", len(cliques['all_cliques'])])
                writer.writerow(["Nombre de clique:", cliques['clique_number']])
                writer.writerow([])
                writer.writerow(["Cliques les plus grandes:"])
                for i, clique in enumerate(cliques['largest_cliques'][:5]):
                    writer.writerow([f"Clique {i+1}:", str(clique)])
            elif analysis_type == 'k_cores':
                writer.writerow(["ANALYSE DES K-CORES"])
                k_cores = self.analyzer.analysis_results['k_cores']
                writer.writerow(["Noeud", "Valeur k-core"])
                for node, k_value in sorted(k_cores.items()):
                    writer.writerow([node, k_value])
    
    def export_graph(self, graph_type, export_dir):
        fig = Figure(figsize=(10, 8), dpi=300)
        ax = fig.add_subplot(111)
        
        if graph_type == 'network_graph':
            pos = nx.spring_layout(self.analyzer.G, seed=42)
            node_colors = [self.analyzer.node_colors[node] for node in self.analyzer.G.nodes()]
            nx.draw(self.analyzer.G, pos, ax=ax, with_labels=True, 
                    node_color=node_colors, node_size=700, font_size=10, 
                    edge_color='gray', alpha=0.7, font_weight='bold')
            ax.set_title("Réseau du Club de Karaté de Zachary", fontsize=16, fontweight='bold')
            ax.axis('off')
            filename = f"{export_dir}/graphe_reseau.png"
        elif graph_type == 'degree_graph':
            distribution = self.analyzer.analysis_results['degree_distribution']
            degrees = list(distribution.keys())
            counts = list(distribution.values())
            ax.bar(degrees, counts, alpha=0.7, color='#3498DB', edgecolor='#2980B9')
            ax.set_xlabel('Degré', fontsize=12, fontweight='bold')
            ax.set_ylabel('Nombre de nœuds', fontsize=12, fontweight='bold')
            ax.set_title('Distribution des Degrés', fontsize=16, fontweight='bold')
            
            # Set x-axis to show integer values for degrees
            ax.set_xticks(degrees)
            ax.set_xticklabels(degrees)
            
            # Set y-axis to show integer values for counts
            max_count = max(counts)
            y_ticks = range(0, max_count + 1)
            ax.set_yticks(y_ticks)
            ax.set_yticklabels(y_ticks)
            
            ax.grid(True, alpha=0.3)
            filename = f"{export_dir}/distribution_degres.png"

        fig.savefig(filename, bbox_inches='tight', dpi=300)
    
    def export_matrix(self, export_dir):
        filename = f"{export_dir}/matrice_adjacence.csv"
        n = len(self.analyzer.G.nodes())
        current_matrix = self.analyzer.adjacency_matrix[:n, :n]
        np.savetxt(filename, current_matrix, delimiter=',', fmt='%d')
    
    def add_node(self):
        new_node = len(self.analyzer.G.nodes())
        self.analyzer.G.add_node(new_node)
        self.analyzer.node_colors[new_node] = '#3498DB'
        self.analyzer.run_complete_analysis()
        self.refresh_current_display()
        messagebox.showinfo("Succès", f"Nœud {new_node} ajouté avec succès!")
    
    def remove_node(self):
        def remove():
            try:
                node = int(entry.get())
                if node in self.analyzer.G.nodes():
                    self.analyzer.G.remove_node(node)
                    if node in self.analyzer.node_colors:
                        del self.analyzer.node_colors[node]
                    self.analyzer.run_complete_analysis()
                    self.refresh_current_display()
                    dialog.destroy()
                    messagebox.showinfo("Succès", f"Nœud {node} supprimé avec succès!")
                else:
                    messagebox.showerror("Erreur", "Nœud non existant!")
            except ValueError:
                messagebox.showerror("Erreur", "Veuillez entrer un numéro valide!")
        
        dialog = tk.Toplevel(self.root)
        dialog.title("Supprimer Nœud")
        dialog.geometry("300x150")
        tk.Label(dialog, text="Numéro du nœud à supprimer:").pack(pady=10)
        entry = tk.Entry(dialog, font=('Segoe UI', 12))
        entry.pack(pady=5)
        tk.Button(dialog, text="Supprimer", command=remove).pack(pady=10)
    
    def add_edge(self):
        def add():
            try:
                node1 = int(entry1.get())
                node2 = int(entry2.get())
                if node1 in self.analyzer.G.nodes() and node2 in self.analyzer.G.nodes():
                    self.analyzer.G.add_edge(node1, node2)
                    self.analyzer.run_complete_analysis()
                    self.refresh_current_display()
                    dialog.destroy()
                    messagebox.showinfo("Succès", f"Arête ({node1}, {node2}) ajoutée avec succès!")
                else:
                    messagebox.showerror("Erreur", "Nœuds non existants!")
            except ValueError:
                messagebox.showerror("Erreur", "Veuillez entrer des numéros valides!")
        
        dialog = tk.Toplevel(self.root)
        dialog.title("Ajouter Arête")
        dialog.geometry("300x200")
        tk.Label(dialog, text="Nœud 1:").pack(pady=5)
        entry1 = tk.Entry(dialog, font=('Segoe UI', 12))
        entry1.pack(pady=5)
        tk.Label(dialog, text="Nœud 2:").pack(pady=5)
        entry2 = tk.Entry(dialog, font=('Segoe UI', 12))
        entry2.pack(pady=5)
        tk.Button(dialog, text="Ajouter", command=add).pack(pady=10)
    
    def remove_edge(self):
        def remove():
            try:
                node1 = int(entry1.get())
                node2 = int(entry2.get())
                if self.analyzer.G.has_edge(node1, node2):
                    self.analyzer.G.remove_edge(node1, node2)
                    self.analyzer.run_complete_analysis()
                    self.refresh_current_display()
                    dialog.destroy()
                    messagebox.showinfo("Succès", f"Arête ({node1}, {node2}) supprimée avec succès!")
                else:
                    messagebox.showerror("Erreur", "Arête non existante!")
            except ValueError:
                messagebox.showerror("Erreur", "Veuillez entrer des numéros valides!")
        
        dialog = tk.Toplevel(self.root)
        dialog.title("Supprimer Arête")
        dialog.geometry("300x200")
        tk.Label(dialog, text="Nœud 1:").pack(pady=5)
        entry1 = tk.Entry(dialog, font=('Segoe UI', 12))
        entry1.pack(pady=5)
        tk.Label(dialog, text="Nœud 2:").pack(pady=5)
        entry2 = tk.Entry(dialog, font=('Segoe UI', 12))
        entry2.pack(pady=5)
        tk.Button(dialog, text="Supprimer", command=remove).pack(pady=10)
    
    def reset_network(self):
        self.analyzer = KarateClubAnalyzer()
        self.analyzer.run_complete_analysis()
        self.refresh_current_display()
        messagebox.showinfo("Succès", "Réseau réinitialisé avec succès!")
    
    def refresh_current_display(self):
        if self.current_display == "graph":
            self.show_graph()
        elif self.current_display == "degree_distribution":
            self.show_degree_distribution()
        elif self.current_display == "analysis":
            self.show_analysis()
        elif self.current_display == "modifier":
            self.show_modifier()
        elif self.current_display == "exporter":
            self.show_exporter()



In [ ]:
def main():
    # If CustomTkinter is available, set appearance
    if USE_CTK:
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
    root = tk.Tk()
    app = ModernKarateClubGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()